In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
model = SentenceTransformer('jhgan/ko-sroberta-multitask')      # 한글문장 임베딩BERT모델

In [4]:
sentences = ['안녕하세요?', '한글 문장 임베딩을 위한 BERT 모델입니다.']
embeddings = model.encode(sentences)        
print(embeddings)

[[-0.37510458 -0.7733839   0.5927711  ...  0.57923526  0.3268346
  -0.65089655]
 [-0.03811135  0.20955126  0.02041095 ...  0.25595438  0.27676168
   0.06661468]]


In [5]:
df = pd.read_csv('data/부동산용어3200_data.csv')
df.head(3)

,분류,유저,챗봇
0,용어,제2종 일반주거지역,"제2종 일반주거지역\n용도지역의 주거지역 중 일반주거지역의 하나로, 중층주택을 중심..."
1,용어,제3종 일반주거지역,"제3종 일반주거지역\n용도지역의 주거지역 중 일반주거지역의 하나로, 중 · 고층주택..."
2,용어,제1종 일반주거지역,"제1종 일반주거지역\n용도지역의 주거지역 중 일반주거지역의 하나로, 저층주택을 중심..."


In [26]:
model.encode(df.유저[0])

array([-8.19176361e-02, -2.99926132e-01,  5.33842266e-01, -5.54296374e-01,
        5.27549267e-01, -4.73842263e-01, -1.27218977e-01, -6.81212246e-02,
        2.56909966e-01, -4.95992184e-01,  4.80895728e-01,  2.74099380e-01,
       -4.84179914e-01,  2.26130784e-01, -1.61423326e-01,  4.48896945e-01,
        1.29693672e-01, -3.59812289e-01,  4.22749609e-01, -4.27382678e-01,
        2.68361747e-01,  1.39616877e-01,  8.44628751e-01,  1.34372246e-02,
        1.48276225e-01,  4.72740918e-01, -6.40568078e-01,  8.64712894e-02,
        1.05302833e-01, -2.19645109e-02,  6.97867572e-02, -1.94671512e-01,
        2.11358070e-01, -5.63689828e-01,  1.61728024e-01,  5.04748523e-01,
       -6.45821035e-01, -8.21957886e-01,  9.53164220e-01, -1.76519811e-01,
        2.07900897e-01, -5.94553053e-01, -2.40025550e-01,  5.35312831e-01,
        2.51112461e-01, -1.07206851e-01, -8.02093744e-01,  5.54598272e-01,
        4.28610248e-03,  7.01841116e-01,  1.05079913e+00, -5.75812697e-01,
       -6.47186041e-01, -

In [27]:
df['embedding'] = pd.Series([[]] * len(df))     # dummy
df.shape

(3200, 4)

In [28]:
df.embedding = df.유저.map(lambda x: list(model.encode(x)))
df.head(3)

,분류,유저,챗봇,embedding
0,용어,제2종 일반주거지역,"제2종 일반주거지역\n용도지역의 주거지역 중 일반주거지역의 하나로, 중층주택을 중심...","[-0.081917636, -0.29992613, 0.53384227, -0.554..."
1,용어,제3종 일반주거지역,"제3종 일반주거지역\n용도지역의 주거지역 중 일반주거지역의 하나로, 중 · 고층주택...","[0.0842627, -0.5216369, 0.37226632, -0.7002009..."
2,용어,제1종 일반주거지역,"제1종 일반주거지역\n용도지역의 주거지역 중 일반주거지역의 하나로, 저층주택을 중심...","[-0.030305676, -0.43911374, 0.5236608, -0.6437..."


In [29]:
df.to_csv('data/부동용어3200_임베딩_data.csv', index=False)

In [30]:
import json
wdf = pd.read_csv('data/부동용어3200_임베딩_data.csv')
wdf.embedding = wdf.embedding.apply(json.loads)
wdf.head(3)

,분류,유저,챗봇,embedding
0,용어,제2종 일반주거지역,"제2종 일반주거지역\n용도지역의 주거지역 중 일반주거지역의 하나로, 중층주택을 중심...","[-0.081917636, -0.29992613, 0.53384227, -0.554..."
1,용어,제3종 일반주거지역,"제3종 일반주거지역\n용도지역의 주거지역 중 일반주거지역의 하나로, 중 · 고층주택...","[0.0842627, -0.5216369, 0.37226632, -0.7002009..."
2,용어,제1종 일반주거지역,"제1종 일반주거지역\n용도지역의 주거지역 중 일반주거지역의 하나로, 저층주택을 중심...","[-0.030305676, -0.43911374, 0.5236608, -0.6437..."


In [31]:
text = '전세'
embedding = model.encode(text)

In [32]:
wdf['유사도'] = wdf.embedding.map(lambda x: cosine_similarity([embedding],[x]).squeeze())

In [33]:
wdf.유사도.idxmax()

206

In [35]:
wdf.챗봇[wdf.유사도.idxmax()]

'전세\n전세란 집주인이 세 들어 사는 사람에게 집의 방 한 칸이나 집 전체를 빌려주고, 그 대가로 받는 돈이지만 세 사는 사람이 집을 나갈 때는 돌려주어야 하는 돈을 말한다. 사는 집에만 쓰는 말이며, 상가나 빌딩 등에는 쓰지 않는다. 전세는 주거관습으로 생겨 하나의 물권으로 발달된 것으로 한국에만 있는 특이한 주택 임대차 거래관행이다.이것은 집이 있는 사람과 집을 살 능력이 부족한 사람 간에 상호 이익이 부합하여 관행이 된 것이다. 특히 산업화사회에 접어들어 농촌인구의 급격한 도시진입은 주택 부족문제를 발생시키고 수급의 불균형을 초래하였으며, 도시에서 집 살 능력이 부족한 농촌 사람들은 셋집을 얻어야 했으며, 전세는 지속적으로 관행이 되었다.또 몫 돈의 전세금을 받는 사람은 금융권에서 대출받는 것이 어려웠으므로, 이것을 이용해 부족한 집 값을 채워 집을 사는데 활용하였다. 이처럼 이 거래관행은 세를 주는 자와 세를 얻는 자간에 자연스럽게 발전되었다. 이 전세관행은 집주인인 임대인과 세사는 임차인 모두에게 장점이 있다.임대인 입장에서 전세관행은 자금조달 수단으로 활용된다. 앞에 말한 것처럼 집살 때 부족한 자금을 이 전세자금으로 충당할 수 있고, 사업자금으로 활용하거나 금융기관에 맡겨 이자를 받을 수 있다. 임차인 입자에서는 강제 저축수단이 된다. 셋집을 얻을 때는 목돈이 들기 때문에 어떻게든 아끼고 절약하여 전셋돈을 마련한다.또 전세금으로 주고 나면 쓸 수도 없어서 돈이 낭비되지 않는다. 이처럼 강제 저축의 성격이 있는 이 자금은 나중에 내 집 마련자금으로 활용할 수 있다. 이러한 관행은 외환위기 이후 경제가 침체되면서 이자율이 낮아지자 셋 집주인들은 셋 돈을 활용하기 어려워 전세금을 보증금 일부, 월세 일부로 환산하여 받기 시작하였으며, 또 세입자도 목돈을 적게 들이는 이 방식을 받아들이게 되었다.그래서 현재는 전세관행이 보증금부 월세 관행으로 전이되는 과정에 있다. 또 서구나 일본처럼 보증금 없이 월세만 내고 임대차하는 관행도 나타나 이용되고 있다. 비

In [40]:
text = '중개'
embedding = model.encode(text)
wdf['유사도'] = wdf.embedding.map(lambda x: cosine_similarity([embedding], [x]).squeeze())
answer = wdf.챗봇[wdf.유사도.idxmax()]
answer

'중개\n거래당사자간의 부동산 등의 매매, 교환, 임대차, 기타 권리의 득실, 변경에 관한 행위를 알선하는 것을 말한다. 공인중개사법상의 중개란 토지, 건축물 그 밖의 토지정착물과 그 밖의 대통령령이 정하는 재산권 및 물건을 거래당사자간의 매매 · 교환 · 임대차 그 밖의 권리의 득실변경에 관한 행위를 알선하는 것을 말한다.'